<a href="https://colab.research.google.com/github/laurence-lin/Retail-Store-Location-Ranking/blob/master/Final_Project_DataCollection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
import tensorflow.keras as keras
import sklearn
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder

import gc
import os
import requests

!pip install folium
!pip install geopy

import folium
from geopy.geocoders import Nominatim

from google.colab import files

pd.set_option('display.max_columns', 500)


Data Feautures in Other user cases:

Collectyed by Foursquare API:

10 * 10 km square in New York City. Which contains:

37422 geo-tagged venues,

46855 user check-in,

620932 total check-in along 5 months.

Collected by Twitter API:

The analyze target contains 3 retail stores:

186 Starbucks, 210174 check-ins

104 Duckin-Donuts, 26955 check-ins

66 Mcdonald's, 15014 check-ins.




In [ ]:
# CLIENT settings
CLIENT_ID = 'client_id'
CLIENT_SECRET = 'client_secret'
VERSION = '20200603'

print('Foursquare parameters assigned.')

Foursquare parameters assigned.


###Collect data from New York

In [ ]:
import geopy

'''
site = 'New York'

# Coordinates of New York center 
latitude = 40.77
longitude = -73.98
print(latitude)
print(longitude)
'''

site = 'Toronto'
latitude = 43.756
longitude = -79.441

In [ ]:
files.upload()
df_to_data = pd.read_csv('gmap_to_MC.csv', index_col = False, encoding = 'iso-8859-1')
df_to_data.head()

KeyboardInterrupt: ignored

In [ ]:
RADIUS = 20000
LIMIT = 200
query = "Mcdonald's"

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}&offset=0'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT,
    query
)

print(url)
result = requests.get(url).json()
print('Get the result. ')
print('Total results: ', result['response']['totalResults'])

In [ ]:
item = result['response']['groups'][0]['items']
# Collect Mcdonald's data from New York 10 km range
col = ['name', 'latitude', 'longitude', 'venue_id', 'city']
df_ny = pd.DataFrame(columns = col)
for index in range(len(item)):
  lat = item[index]['venue']['location']['lat']
  lng = item[index]['venue']['location']['lng']
  ven_id = item[index]['venue']['id']
  city = item[index]['venue']['location']['city']
  name = item[index]['venue']['name']
  df_ny.loc[index] = [name, lat, lng, ven_id, city]
  
print('Current shape: ', df_ny.shape)
df_ny.head()

In [ ]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}&offset=50'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT,
    query
)

print(url)
result = requests.get(url).json()
print('Get the result. ')
print('Total results: ', result['response']['totalResults'])

In [ ]:
item = result['response']['groups'][0]['items']
# Collect Mcdonald's data from New York 10 km range

for index in range(len(item)):
  lat = item[index]['venue']['location']['lat']
  lng = item[index]['venue']['location']['lng']
  ven_id = item[index]['venue']['id']
  city = item[index]['venue']['location']['city']
  name = item[index]['venue']['name']
  df_ny.loc[index + 50] = [name, lat, lng, ven_id, city]
  
print(df_ny.duplicated().any())
print('Current shape: ', df_ny.shape)
print(df_ny.head())

In [ ]:
df_ny.to_csv('df_ny.csv', index = None)

In [ ]:
import geopy

site = 'Toronto'

locator = Nominatim(user_agent = 'laurence_user')
location = locator.geocode(site)
print(location.latitude)
print(location.longitude)
latitude = location.latitude
longitude = location.longitude


In [ ]:
RADIUS = 10000
LIMIT = 200
offset = 1
query = "Mcdonald's"

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT,
    query
)


result = requests.get(url).json()
print('Get the result. ')

In [ ]:
result['response']['totalResults']

In [ ]:
item = result['response']['groups'][0]['items']
# Collect Mcdonald's data from New York 10 km range
df_to = pd.DataFrame(columns = col)
for index in range(len(item)):
  lat = item[index]['venue']['location']['lat']
  lng = item[index]['venue']['location']['lng']
  ven_id = item[index]['venue']['id']
  city = item[index]['venue']['location']['city']
  df_to.loc[index] = [lat, lng, ven_id, city]

df_to.head()

In [ ]:
df_to.shape

###Collect number of comments from Google Map: Only 60 places for each city search (Limit by Google Place API)

In [ ]:
col = ['name', 'latitude', 'longitude', 'ratings', 'comments']
df_gmap = pd.DataFrame(columns = col)
df_gmap

In [ ]:
import requests
api_key = 'API_KEY'
radius = 10000
keyword = 'Mcdonald'
fields = 'geometry,user_ratings_total,rating,name,place_id'
latitude = 40.77
longitude = -73.98

In [ ]:
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?key={}&query={}&location={},{}&radius={}".format(
    api_key,
    keyword,
    latitude,
    longitude,
    radius
    )
                                                                                                                       
print(url)
print(type(url))
result = requests.get(url).json()

for i in range(len(result['results'])):
  lat = result['results'][i]['geometry']['location']['lat']
  lng = result['results'][i]['geometry']['location']['lng']
  name = result['results'][i]['name']
  rating = result['results'][i]['rating']
  total_rating = result['results'][i]['user_ratings_total']
  df_gmap.loc[i] = [name, lat, lng, rating, total_rating]


In [ ]:
# Get next 20 results
page_token = result['next_page_token']

url = "https://maps.googleapis.com/maps/api/place/textsearch/json?key={}&query={}&location={},{}&radius={}&pagetoken={}".format(
    api_key,
    keyword,
    latitude,
    longitude,
    radius,
    page_token
    )
                                                                                                                       
print(url)
print(type(url))
result = requests.get(url).json()

for i in range(len(result['results'])):
  lat = result['results'][i]['geometry']['location']['lat']
  lng = result['results'][i]['geometry']['location']['lng']
  name = result['results'][i]['name']
  rating = result['results'][i]['rating']
  total_rating = result['results'][i]['user_ratings_total']
  df_gmap.loc[i + 20] = [name, lat, lng, rating, total_rating]

In [ ]:
page_token = result['next_page_token']
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?key={}&query={}&location={},{}&radius={}&pagetoken={}".format(
    api_key,
    keyword,
    latitude,
    longitude,
    radius,
    page_token
    )
                                                                                                                       
print(url)
print(type(url))
result = requests.get(url).json()

for i in range(len(result['results'])):
  lat = result['results'][i]['geometry']['location']['lat']
  lng = result['results'][i]['geometry']['location']['lng']
  name = result['results'][i]['name']
  rating = result['results'][i]['rating']
  total_rating = result['results'][i]['user_ratings_total']
  df_gmap.loc[i + 40] = [name, lat, lng, rating, total_rating]

In [ ]:
print(df_gmap.duplicated().any())
print(df_gmap.shape)
print(df_gmap.head())

In [ ]:
df_gmap.to_csv('gmap_ny_MC.csv', index = None)

In [ ]:
# Dataframe to store Toronto Mcdonald's data
df_gmap_to = pd.DataFrame(columns = df_gmap.columns)
df_gmap_to

In [ ]:
# Get ratings and total_ratings from Toronto Mcdonald's

latitude = 43.651070
longitude = -79.347015
radius = 10000
keyword = 'Mcdonald'

url = "https://maps.googleapis.com/maps/api/place/textsearch/json?key={}&query={}&location={},{}&radius={}".format(
    api_key,
    keyword,
    latitude,
    longitude,
    radius
    )
                                                                                                                       
print(url)
print(type(url))
result = requests.get(url).json()

for i in range(len(result['results'])):
  lat = result['results'][i]['geometry']['location']['lat']
  lng = result['results'][i]['geometry']['location']['lng']
  name = result['results'][i]['name']
  rating = result['results'][i]['rating']
  total_rating = result['results'][i]['user_ratings_total']
  df_gmap_to.loc[i] = [name, lat, lng, rating, total_rating]

print(df_gmap_to.shape)
print(df_gmap_to.head())

In [ ]:
page_token = result['next_page_token']
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?key={}&query={}&location={},{}&radius={}&pagetoken={}".format(
    api_key,
    keyword,
    latitude,
    longitude,
    radius,
    page_token
    )
                                                                                                                       
print(url)
print(type(url))
result = requests.get(url).json()

for i in range(len(result['results'])):
  lat = result['results'][i]['geometry']['location']['lat']
  lng = result['results'][i]['geometry']['location']['lng']
  name = result['results'][i]['name']
  rating = result['results'][i]['rating']
  total_rating = result['results'][i]['user_ratings_total']
  df_gmap_to.loc[i + 20] = [name, lat, lng, rating, total_rating]

print(df_gmap_to.shape)
print(df_gmap_to.duplicated().any())


In [ ]:
page_token = result['next_page_token']
url = "https://maps.googleapis.com/maps/api/place/textsearch/json?key={}&query={}&location={},{}&radius={}&pagetoken={}".format(
    api_key,
    keyword,
    latitude,
    longitude,
    radius,
    page_token
    )
                                                                                                                       
print(url)
print(type(url))
result = requests.get(url).json()

for i in range(len(result['results'])):
  lat = result['results'][i]['geometry']['location']['lat']
  lng = result['results'][i]['geometry']['location']['lng']
  name = result['results'][i]['name']
  rating = result['results'][i]['rating']
  total_rating = result['results'][i]['user_ratings_total']
  df_gmap_to.loc[i + 40] = [name, lat, lng, rating, total_rating]

print(df_gmap_to.shape)
print(df_gmap_to.duplicated().any())

In [ ]:
df_gmap_to.to_csv('gmap_to_MC.csv', index = False)

In [ ]:
df_ny.head()

In [ ]:
from google.colab import files

files.upload()
df_gmap_ny = pd.read_csv('gmap_ny_MC.csv', index_col = False)
df_gmap_ny.head()


In [ ]:
files.upload()
df_ny = pd.read_csv('df_ny.csv', index_col = False)

df_ny.head()

In [ ]:
# Due to the coordinate bias scraped from different API, we need to round the latitude and longitude to same decimal point for later combination
import math

def round_num(num, precision = 0):
  num = math.ceil(num *(10**(precision)) )
  return num/(10**precision)

for i in range(df_ny.shape[0]):
    df_ny.loc[i,'latitude'] = round_num(df_ny.loc[i,'latitude'], 4)
    df_ny.loc[i,'longitude'] = round_num(df_ny.loc[i,'longitude'], 4)

for i in range(df_gmap_ny.shape[0]):
     df_gmap_ny.loc[i, 'latitude'] = round_num(df_gmap_ny.loc[i,'latitude'], 4)
     df_gmap_ny.loc[i, 'longitude'] = round_num(df_gmap_ny.loc[i, 'longitude'], 4)

print('Round coordinates finished. ')

In [ ]:
#Because there are still biases btw same Retail store spots btw Google place API & Foursquare API, we have to manually adjust the longitude
for i in range(len(df_ny)):
  for j in range(len(df_gmap_ny)):
    if abs(df_ny.iloc[i]['latitude'] - df_gmap_ny.iloc[j]['latitude']) <= 0.0001:
        if abs(df_ny.iloc[i]['longitude'] - df_gmap_ny.iloc[j]['longitude']) <= 0.0002:
               df_ny.loc[i, 'latitude'] = df_gmap_ny.loc[j, 'latitude']
               df_ny.loc[i, 'longitude'] = df_gmap_ny.loc[j, 'longitude']

print('Adjustment finished. ')


In [ ]:
df_ny_data = df_ny.merge(df_gmap_ny, how = 'inner', on = ['latitude', 'longitude'])
print(df_ny_data.shape)
print(df_ny_data.head())

In [ ]:


ny_map = folium.Map(location = [latitude, longitude], zoom_start = 12)
ny_map

In [ ]:
for i in range(len(df_ny)):
    folium.CircleMarker(
          [df_ny.iloc[i][ 'latitude'], df_ny.iloc[i]['longitude']],
          radius = 5,
          color = 'red',
          fill = True,
          fill_color = 'red',
          fill_opacity = 0.6,
          popup = str(df_ny.iloc[i]['latitude']) + str(df_ny.iloc[i]['longitude'])
    ).add_to(ny_map)

ny_map

In [ ]:
for i in range(len(df_gmap_ny)):
    folium.Marker(
          [df_gmap_ny.iloc[i][ 'latitude'], df_gmap_ny.iloc[i]['longitude']],
          popup = str(df_gmap_ny.iloc[i]['latitude']) + str(df_gmap_ny.iloc[i]['longitude'])
    ).add_to(ny_map)

ny_map

In [ ]:
for i in range(len(df_ny_data)):
    folium.Marker(
          [df_ny_data.iloc[i][ 'latitude'], df_ny_data.iloc[i]['longitude']],
          popup = str(df_ny_data.iloc[i]['latitude']) + str(df_ny_data.iloc[i]['longitude']),
          icon = folium.Icon(color = 'green')
    ).add_to(ny_map)

ny_map

In [ ]:
df_ny_data.drop('name_y', axis = 1, inplace = True)
df_ny_data.head()

In [ ]:
df_ny_data.to_csv('df_ny_data', index = None)

In [ ]:
# Compute Dense: number of venues arount the store location
RADIUS = 200
LIMIT = 200

for i in range(len(df_ny_data)):
    latitude = df_ny_data.loc[i, 'latitude']
    longitude = df_ny_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          RADIUS, 
          LIMIT,
        )

    result = requests.get(url).json()
    df_ny_data.loc[i, 'Density'] = result['response']['totalResults']


# Compute entropy of neighbor venues 
entropy_list = []
for i in range(df_ny_data.shape[0]):
    latitude = df_ny_data.loc[i, 'latitude']
    longitude = df_ny_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          RADIUS, 
          LIMIT
          )
    
    result = requests.get(url).json()
    item = result['response']['groups'][0]['items']
    venue_type = []
    for i in range(len(item)):
      venue_type.append(item[i]['venue']['categories'][0]['name'])

    venue_count = pd.Series(venue_type).value_counts() #Counts of each category venues arount the retail store
    n_total = result['response']['totalResults']  # total neighbor venues
    entropy = 0
    for i in range(len(venue_count)):
        entropy += (venue_count[i]/n_total)*np.log(venue_count[i]/n_total)

    entropy *= -1
    entropy_list.append(entropy)

df_ny_data['neighbor_entropy'] = entropy_list[:]

# Compute competitiveness: number of same type venues arount the store
category = '4bf58dd8d48988d16e941735' # category Id for Fast Food Restaurant

compet_list = []

for i in range(df_ny_data.shape[0]):
    latitude = df_ny_data.loc[i, 'latitude']
    longitude = df_ny_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          RADIUS, 
          LIMIT,
          category
          )
    
    result = requests.get(url).json()
    n_compet = result['response']['totalResults']  # total neighbor venues
    compet_ratio = - n_compet/df_ny_data.loc[i, 'Density']
    compet_list.append(compet_ratio)

df_ny_data['competitiveness'] = compet_list[:]    

# Add restaurants other than competitors as feature
food = '4d4b7105d754a06374d81259' # category of all types of restaurant nearby

food_venues = []
for i in range(df_ny_data.shape[0]):
    latitude = df_ny_data.loc[i, 'latitude']
    longitude = df_ny_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS, 
            LIMIT,
            food
            )
    
    result = requests.get(url).json()
    total_food = result['response']['totalResults']
    # Excluding Competitors, save only other type restaurants
    item = result['response']['groups'][0]['items']
    for i in range(len(item)):
        if item[i]['venue']['categories'][0]['name'] == 'Fast Food Restaurant':
            total_food -= 1

    food_venues.append(total_food)

df_ny_data['other_food_neighbor'] = food_venues

# Try to normalize the number of other restaurants by total neighbors, like for competitiveness
df_ny_data['food_neighbor_rate'] = df_ny_data['other_food_neighbor'] / df_ny_data['Density']

# Add Residence places for neighbor for estimation of demographic
RADIUS = 200
LIMIT = 500
resident = '4e67e38e036454776db1fb3a'

residence = []
for i in range(df_ny_data.shape[0]):
    latitude = df_ny_data.loc[i, 'latitude']
    longitude = df_ny_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS, 
            LIMIT,
            resident
            )
    
    result = requests.get(url).json()
    total_res = result['response']['totalResults']
    residence.append(total_res)

df_ny_data['residence_venue'] = residence


###Collect data for Toronto

In [ ]:
import geopy


site = 'Toronto'
latitude = 43.7318
longitude = -79.4213

In [ ]:
RADIUS = 20000
LIMIT = 200
query = "Mcdonald's"

url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT,
    query
)

print(url)
result = requests.get(url).json()
print('Get the result. ')
print('Total results: ', result['response']['totalResults'])

https://api.foursquare.com/v2/venues/explore?client_id=02FTVAGTLJ1JCOACKPUG1FYI44KBDYCDVCYBT1BBZRPZ5PI2&client_secret=IPSZRP2ZALOFZI2GCSCDP5UZPQWUDQ44XDTPKU3AQCTYTQPU&v=20200603&ll=43.756,-79.441&radius=20000&limit=200&query=Mcdonald's
Get the result. 
Total results:  41


In [ ]:
item = result['response']['groups'][0]['items']
# Collect Mcdonald's data from New York 10 km range
col = ['name', 'latitude', 'longitude', 'venue_id', 'city']
df_to = pd.DataFrame(columns = col)
for index in range(len(item)):
  lat = item[index]['venue']['location']['lat']
  lng = item[index]['venue']['location']['lng']
  ven_id = item[index]['venue']['id']
  city = item[index]['venue']['location']['city']
  name = item[index]['venue']['name']
  df_to.loc[index] = [name, lat, lng, ven_id, city]
  
print('Current shape: ', df_to.shape)
df_to.head()

Current shape:  (41, 5)


,name,latitude,longitude,venue_id,city
0,McDonald's,43.738386,-79.606120,4afafacff964a5200d1a22e3,Etobicoke
1,McDonald's,43.769853,-79.370652,4af41aa4f964a52016f021e3,North York
2,McDonald's,43.704919,-79.452238,52ed1927498e2e54dfbcc99b,Toronto
3,McDonald's,43.758028,-79.465392,4b9277e5f964a520e0fb33e3,Toronto
4,McDonald's,43.774045,-79.530902,4ae46049f964a520329a21e3,Vaughan


In [ ]:
RADIUS = 20000
LIMIT = 100
query = "Mcdonald's"

url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&query={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    RADIUS, 
    LIMIT,
    query
)

print(url)
result = requests.get(url).json()
print('Get the result. ')
print('Total results: ', len(result['response']['venues']))

https://api.foursquare.com/v2/venues/search?client_id=02FTVAGTLJ1JCOACKPUG1FYI44KBDYCDVCYBT1BBZRPZ5PI2&client_secret=IPSZRP2ZALOFZI2GCSCDP5UZPQWUDQ44XDTPKU3AQCTYTQPU&v=20200603&ll=43.756,-79.441&radius=20000&limit=100&query=Mcdonald's
Get the result. 
Total results:  50


In [ ]:
item = result['response']['venues']
# Collect Mcdonald's data from New York 10 km range
col = ['name', 'latitude', 'longitude', 'venue_id', 'city']
df_to_2 = pd.DataFrame(columns = col)
for index in range(len(item)):
  lat = item[index]['location']['lat']
  lng = item[index]['location']['lng']
  ven_id = item[index]['id']
  city = item[index]['location']['city']
  name = item[index]['name']
  df_to_2.loc[index] = [name, lat, lng, ven_id, city]
  
print('Current shape: ', df_to_2.shape)


Current shape:  (50, 5)


In [ ]:

df_to_3 = df_to.merge(df_to_2, how = 'outer', on = ['latitude', 'longitude'])
df_to_3.drop_duplicates(inplace = True)
print(df_to_3.shape)


(60, 8)


In [ ]:
# Due to the coordinate bias scraped from different API, we need to round the latitude and longitude to same decimal point for later combination
import math

def round_num(num, precision = 0):
  num = math.ceil(num *(10**(precision)) )
  return num/(10**precision)

for i in range(df_to_3.shape[0]):
    df_to_3.loc[i,'latitude'] = round_num(df_to_3.loc[i,'latitude'], 3)
    df_to_3.loc[i,'longitude'] = round_num(df_to_3.loc[i,'longitude'], 3)

for i in range(df_gmap_to.shape[0]):
     df_gmap_to.loc[i, 'latitude'] = round_num(df_gmap_to.loc[i,'latitude'], 3)
     df_gmap_to.loc[i, 'longitude'] = round_num(df_gmap_to.loc[i, 'longitude'], 3)

print('Round coordinates finished. ')

Round coordinates finished. 


In [ ]:
df_to_data = df_to_3.merge(df_gmap_to, how = 'inner', on = ['latitude', 'longitude'])
df_to_data.shape

(27, 11)

In [ ]:
map_to = folium.Map(zoom_start = 12, location = [latitude, longitude])
map_to

In [ ]:
for i in range(len(df_to_3)):
    folium.CircleMarker(
          [df_to_3.iloc[i][ 'latitude'], df_to_3.iloc[i]['longitude']],
          radius = 5,
          color = 'red',
          fill = True,
          fill_color = 'red',
          fill_opacity = 0.6,
          popup = str(df_to_3.iloc[i]['latitude']) + str(df_to_3.iloc[i]['longitude'])
    ).add_to(map_to)

folium.Circle(
    location = [latitude, longitude],
    radius = 20000
).add_to(map_to)

map_to

In [ ]:
for i in range(len(df_gmap_to)):
    folium.Marker(
          [df_gmap_to.iloc[i][ 'latitude'], df_gmap_to.iloc[i]['longitude']],
          popup = str(df_gmap_to.iloc[i]['latitude']) + str(df_gmap_to.iloc[i]['longitude'])
    ).add_to(map_to)

map_to

In [ ]:
for i in range(len(df_to_data)):
    folium.Marker(
          [df_to_data.iloc[i][ 'latitude'], df_to_data.iloc[i]['longitude']],
          popup = str(df_to_data.iloc[i]['latitude']) + str(df_gmap_to.iloc[i]['longitude']),
          icon = folium.Icon(color = 'green')
    ).add_to(map_to)

map_to

In [ ]:
df_to_data.to_csv('df_to_data.csv', index = None)

In [ ]:
# Compute Dense: number of venues arount the store location
RADIUS = 200
LIMIT = 200

for i in range(len(df_to_data)):
    latitude = df_to_data.loc[i, 'latitude']
    longitude = df_to_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          RADIUS, 
          LIMIT,
        )

    result = requests.get(url).json()
    df_to_data.loc[i, 'Density'] = result['response']['totalResults']


# Compute entropy of neighbor venues 
entropy_list = []
for i in range(df_to_data.shape[0]):
    latitude = df_to_data.loc[i, 'latitude']
    longitude = df_to_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          RADIUS, 
          LIMIT
          )
    
    result = requests.get(url).json()
    item = result['response']['groups'][0]['items']
    venue_type = []
    for i in range(len(item)):
      venue_type.append(item[i]['venue']['categories'][0]['name'])

    venue_count = pd.Series(venue_type).value_counts() #Counts of each category venues arount the retail store
    n_total = result['response']['totalResults']  # total neighbor venues
    entropy = 0
    for i in range(len(venue_count)):
        entropy += (venue_count[i]/n_total)*np.log(venue_count[i]/n_total)

    entropy *= -1
    entropy_list.append(entropy)

df_to_data['neighbor_entropy'] = entropy_list[:]

# Compute competitiveness: number of same type venues arount the store
category = '4bf58dd8d48988d16e941735' # category Id for Fast Food Restaurant

compet_list = []

for i in range(df_to_data.shape[0]):
    latitude = df_to_data.loc[i, 'latitude']
    longitude = df_to_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
          CLIENT_ID, 
          CLIENT_SECRET, 
          VERSION, 
          latitude, 
          longitude, 
          RADIUS, 
          LIMIT,
          category
          )
    
    result = requests.get(url).json()
    n_compet = result['response']['totalResults']  # total neighbor venues
    compet_ratio = - n_compet/df_to_data.loc[i, 'Density']
    compet_list.append(compet_ratio)

df_to_data['competitiveness'] = compet_list[:]    

# Add restaurants other than competitors as feature
food = '4d4b7105d754a06374d81259' # category of all types of restaurant nearby

food_venues = []
for i in range(df_to_data.shape[0]):
    latitude = df_to_data.loc[i, 'latitude']
    longitude = df_to_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS, 
            LIMIT,
            food
            )
    
    result = requests.get(url).json()
    total_food = result['response']['totalResults']
    # Excluding Competitors, save only other type restaurants
    item = result['response']['groups'][0]['items']
    for i in range(len(item)):
        if item[i]['venue']['categories'][0]['name'] == 'Fast Food Restaurant':
            total_food -= 1

    food_venues.append(total_food)

df_to_data['other_food_neighbor'] = food_venues

# Try to normalize the number of other restaurants by total neighbors, like for competitiveness
df_to_data['food_neighbor_rate'] = df_to_data['other_food_neighbor'] / df_to_data['Density']

# Add Residence places for neighbor for estimation of demographic
RADIUS = 200
LIMIT = 500
resident = '4e67e38e036454776db1fb3a'

residence = []
for i in range(df_to_data.shape[0]):
    latitude = df_to_data.loc[i, 'latitude']
    longitude = df_to_data.loc[i, 'longitude']
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&categoryId={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            latitude, 
            longitude, 
            RADIUS, 
            LIMIT,
            resident
            )
    
    result = requests.get(url).json()
    total_res = result['response']['totalResults']
    residence.append(total_res)

df_to_data['residence_venue'] = residence


In [ ]:
df_to_data['residence'] = 0

In [ ]:
df_to_data.to_csv('df_to_feature.csv', index = None)